In [3]:
import cv2
import utils
import pickle
import ml_metrics

In [4]:
# Import queries from dataset 1

query_path = 'qsd1_w1'
queries_1 = utils.import_images(query_path)

In [46]:
# Import queries from dataset 2

query_path = 'qsd2_w1'
queries_2 = utils.import_images(query_path)

In [6]:
# Import BBDD

bbdd_path = 'BBDD'
bbdd = utils.import_images(bbdd_path)

In [32]:
# Import GT values for both datasets

gt = pickle.load(open('./qsd1_w1/gt_corresps.pkl', 'rb', buffering=0), encoding='bytes')
gt_2 = pickle.load(open('./qsd2_w1/gt_corresps.pkl', 'rb', buffering=0), encoding='bytes')

## Calculate based on HSV - Method 2

In [8]:
# Convert images to HSV
hsv_queries = utils.convert_color_space(queries_1, cv2.COLOR_BGR2HSV)
hsv_queries = utils.resize_images(hsv_queries, (1024,1024))
hsv_queries = sorted(hsv_queries.items())
hsv_queries = dict(hsv_queries)

hsv_queries_2 = utils.convert_color_space(queries_2, cv2.COLOR_BGR2HSV)
hsv_queries_2 = utils.resize_images(hsv_queries_2, (1024,1024))
hsv_queries_2 = sorted(hsv_queries_2.items())
hsv_queries_2 = dict(hsv_queries_2)

hsv_bbdd = utils.convert_color_space(bbdd, cv2.COLOR_BGR2HSV)
hsv_bbdd = utils.resize_images(hsv_bbdd, (1024,1024))
hsv_bbdd = sorted(hsv_bbdd.items())
hsv_bbdd = dict(hsv_bbdd)

In [9]:
# Calculate HSV histogram
bins = 48

hist_queries = utils.calculate_histograms(hsv_queries, bins, None, bw=False)
hist_bbdd = utils.calculate_histograms(hsv_bbdd, bins, None, bw=False)

In [10]:
# Calculate HSV histogram in number of blocks

blocks = 32

hist_blocks_bbdd = utils.calculate_histograms(hsv_bbdd, bins, None, with_blocks=True, blocks_number=blocks)

In [11]:
hist_blocks_queries_1 = utils.calculate_histograms(hsv_queries, bins, None, with_blocks=True, blocks_number=blocks)

In [12]:
hist_blocks_queries_2 = utils.calculate_histograms(hsv_queries_2, bins, None, with_blocks=True, blocks_number=blocks)

In [13]:
# Get the best k results for the intersection each query image without blocks and all BBDD for the first dataset

result_queries = []

for key_query in queries_1.keys():
    hist_query = hist_queries[key_query]
    result_queries.append(utils.intersection_per_query(hist_query, hist_bbdd, 10))

In [14]:
# Get the best k results for the intersection each query image with blocks and all BBDD for the first dataset

result_queries_with_blocks = []

for key_query in queries_1.keys():
    hist_query = hist_blocks_queries_1[key_query]
    result_queries_with_blocks.append(utils.intersection_per_query(hist_query, hist_blocks_bbdd, 10))

In [15]:
# Result for DS 1 without blocks

eval = ml_metrics.mapk(gt, result_queries, 1)
eval

0.4666666666666667

In [16]:
# Result for DS 1 with blocks

eval = ml_metrics.mapk(gt, result_queries_with_blocks, 1)
eval

0.8333333333333334

In [17]:
# Result for DS 1 without blocks

eval = ml_metrics.mapk(gt, result_queries, 5)
eval

0.5233333333333333

In [18]:
# Result for DS 1 with blocks

eval = ml_metrics.mapk(gt, result_queries_with_blocks, 5)
eval

0.861111111111111

In [19]:
# Get the best k results for the intersection each query image with blocks and all BBDD for the second dataset

result_queries_with_blocks = []

for key_query in queries_2.keys():
    hist_query = hist_blocks_queries_2[key_query]
    result_queries_with_blocks.append(utils.intersection_per_query(hist_query, hist_blocks_bbdd, 10))

In [20]:
# Result for DS 2 with blocks

eval = ml_metrics.mapk(gt_2, result_queries_with_blocks, 1)
eval

0.3

In [21]:
# Result for DS 2 with blocks

eval = ml_metrics.mapk(gt_2, result_queries_with_blocks, 5)
eval

0.35944444444444446

## Calculate based on CieLab - Method 1

In [22]:
# Convert images to CieLab
lab_queries = utils.convert_color_space(queries_1, cv2.COLOR_BGR2Lab)
lab_queries = sorted(lab_queries.items())
lab_queries = dict(lab_queries)

lab_bbdd = utils.convert_color_space(bbdd, cv2.COLOR_BGR2Lab)

In [23]:
# Calculate histograms
bins = 48

hist_queries_lab = utils.calculate_histograms(lab_queries,bins, query_path, False, True)
hist_bbdd_lab = utils.calculate_histograms(lab_bbdd,bins, bbdd_path ,False,True)

In [24]:
# Get the best k results for the intersection all query images

result_queries = []

for key_query in queries_1.keys():
    hist_query = hist_queries_lab[key_query]
    result_queries.append(utils.intersection_per_query(hist_query, hist_bbdd_lab, 5))

In [25]:
res_1 = ml_metrics.mapk(gt, result_queries, 1)
res_1

0.5

In [26]:
res_5 = ml_metrics.mapk(gt, result_queries, 5)
res_5

0.5888888888888888

# Task 5: For each image in the QS2, remove background

In [47]:
# Calculate masks for the second dataset

masks_query, masked_query = utils.find_mask(queries_2)

In [48]:
# Precision, recall, F1 score

query_path = 'qsd2_w1'
gt_mask = utils.import_images_gt(query_path)

res_mask = utils.eval(gt_mask, masks_query)
res_mask

(0.8016236180482522, 0.9602464293550502, 0.8633473125933117)